In [ ]:
from datetime import datetime, timedelta
import calendar
import boto3
from botocore.exceptions import NoCredentialsError
import pandas as pd
from io import StringIO
from pyspark.sql import SparkSession
import pandas as pd 

In [ ]:
import os
# To work with Amazon S3 storage, set the following variables using your AWS Access Key and Secret Key
# Set the Region to where your files are stored in S3.
access_key = 'abc'
secret_key = 'abc'
# Set the environment variables so boto3 can pick them up later
os.environ['AWS_ACCESS_KEY_ID'] = access_key
os.environ['AWS_SECRET_ACCESS_KEY'] = secret_key
encoded_secret_key = secret_key.replace("/", "%2F")
aws_region = "us-east-2"
# Set this to the name of your bucket where the files are stored
aws_bucket_name = "yelp-review-ss"
mount_name = "s3dataread"

Run this on databricks or EC2 Instance

In [ ]:
# Update the Spark options to work with our AWS Credentials
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3." + aws_region +
".amazonaws.com")

In [ ]:
spark = SparkSession.builder.appName("yelpr").getOrCreate()

# Transforming User

In [ ]:
# Path to Amazon S3 files
yelp_user_file = "s3a://yelp-review-ss/yelp_academic_dataset_user.json"

df_user = spark.read.json(yelp_user_file)

In [ ]:
df_user = df_user.select('user_id','name','review_count','yelping_since','friends','useful','funny','cool','fans')
df_user= df_user.dropDuplicates(["user_id"])

In [ ]:
# this will make it easier to convert to CSV
pd_user = df_user.toPandas()

In [ ]:
pd_user['yelping_since'] = pd.to_datetime(pd_user['yelping_since'])
pd_user.info()

In [ ]:
# File is too big; need to convert to two 
df_user_part1 = pd_user.iloc[:len(pd_user)//2]
df_user_part2 = pd_user.iloc[len(pd_user)//2:]

In [ ]:
# Convert DataFrames to CSV-formatted strings
csv_buffer_part1 = StringIO()
df_user_part1.to_csv(csv_buffer_part1, index=False)

csv_buffer_part2 = StringIO()
df_user_part2.to_csv(csv_buffer_part2, index=False)


Store in S3

In [ ]:
# Upload to S3
s3_key_part1 = 'clean/dim_user1.csv'
s3_key_part2 = 'clean/dim_user2.csv'

s3_client = boto3.client('s3')
s3_client.put_object(Body=csv_buffer_part1.getvalue(), Bucket=aws_bucket_name, Key=s3_key_part1)
s3_client.put_object(Body=csv_buffer_part2.getvalue(), Bucket=aws_bucket_name, Key=s3_key_part2)

# Transforming Review (biggest file)

In [ ]:
yelp_review_file = "s3a://yelp-review-ss/yelp_academic_dataset_review.json"

df_review = spark.read.json(yelp_review_file)

In [ ]:
df_review = df_review.drop('text')
df_review = df_review.dropDuplicates(["review_id"])

In [ ]:
pandas_df = df_review.toPandas()

In [ ]:
pandas_df['date'] = pd.to_datetime(pandas_df['date'])
pandas_df.info()

In [ ]:
column_order = ['review_id','user_id','business_id','date','useful','funny','cool','stars']
pandas_df = pandas_df[column_order]